In [1]:
import torch
from torch import nn
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
with open('/kaggle/input/small-dataset/small_dataset.txt','r', encoding='utf-8') as f:
    data=f.readlines()
    print(data)

['To Sherlock Holmes she is always _the_ woman. I have seldom heard him\n', 'mention her under any other name. In his eyes she eclipses and\n', 'predominates the whole of her sex. It was not that he felt any emotion\n', 'akin to love for Irene Adler. All emotions, and that one particularly,\n', 'were abhorrent to his cold, precise but admirably balanced mind. He\n', 'was, I take it, the most perfect reasoning and observing machine that\n', 'the world has seen, but as a lover he would have placed himself in a\n', 'false position. He never spoke of the softer passions, save with a gibe\n', 'and a sneer. They were admirable things for the observer—excellent for\n', 'drawing the veil from men’s motives and actions. But for the trained\n', 'reasoner to admit such intrusions into his own delicate and finely\n', 'adjusted temperament was to introduce a distracting factor which might\n', 'throw a doubt upon all his mental results. Grit in a sensitive\n', 'instrument, or a crack in one of his o

In [2]:
f=open('/kaggle/input/dataset/Adventure_of_Sherlock_Holmes.txt','r', encoding='utf-8')
data=f.readlines()

Now we will take all the lines into a single list.

In [3]:
def preprocess_text_file(text:list):
    processed_sentence_ls=[]
    for sentence in text:
        tokens=[]
        
        for word in word_tokenize(sentence):
            
            if word.isalpha():
                tokens.append(word.lower())
        processed_sentence_ls.append(' '.join(tokens))
    
    
    return processed_sentence_ls
#Veryfying the function.
preprocess_text_file(['hello I"m Saikat','Who are you?'])

['hello i m saikat', 'who are you']

In [4]:
def extract_corpus_vocab(text_ls:list):
    '''Extracts the total vocab of the whole text corpus.'''
    text=' '.join(text_ls)
    vocab_ls=[]
    for word in word_tokenize(text):   #word_tokenize returns a list of the words in the text corpus/sentence.
        
        if word not in vocab_ls:
            vocab_ls.append(word)
        
    return vocab_ls,len(vocab_ls)     

#Checking the function.
extract_corpus_vocab(preprocess_text_file(['hello I"m Saikat','Who are you?']))    

(['hello', 'i', 'm', 'saikat', 'who', 'are', 'you'], 7)

In [5]:
#Applying the functions on the dataset
processed_data=preprocess_text_file(data)
vocab,vocab_length=extract_corpus_vocab(processed_data)
print(vocab_length)

313


In [6]:
print(vocab)

['to', 'sherlock', 'holmes', 'she', 'is', 'always', 'woman', 'i', 'have', 'seldom', 'heard', 'him', 'mention', 'her', 'under', 'any', 'other', 'name', 'in', 'his', 'eyes', 'eclipses', 'and', 'predominates', 'the', 'whole', 'of', 'sex', 'it', 'was', 'not', 'that', 'he', 'felt', 'emotion', 'akin', 'love', 'for', 'irene', 'adler', 'all', 'emotions', 'one', 'particularly', 'were', 'abhorrent', 'cold', 'precise', 'but', 'admirably', 'balanced', 'mind', 'take', 'most', 'perfect', 'reasoning', 'observing', 'machine', 'world', 'has', 'seen', 'as', 'a', 'lover', 'would', 'placed', 'himself', 'false', 'position', 'never', 'spoke', 'softer', 'passions', 'save', 'with', 'gibe', 'sneer', 'they', 'admirable', 'things', 'drawing', 'veil', 'from', 'men', 's', 'motives', 'actions', 'trained', 'reasoner', 'admit', 'such', 'intrusions', 'into', 'own', 'delicate', 'finely', 'adjusted', 'temperament', 'introduce', 'distracting', 'factor', 'which', 'might', 'throw', 'doubt', 'upon', 'mental', 'results', 'gr

In [7]:
def vocab_index_creator(vocab:list):
    vocab=sorted(vocab)  #sorted returns a sorted() list but list.sort() doesn't return anything.
    vocab_dict={}
    for idx,word in enumerate(vocab):
            vocab_dict[word]=idx
    
    return vocab_dict

vocab_index_creator(['hello', 'I"m Saikat','Who',' are',' you?'])


{' are': 0, ' you?': 1, 'I"m Saikat': 2, 'Who': 3, 'hello': 4}

In [8]:
vocab_idx_dict=vocab_index_creator(vocab)
print(vocab_idx_dict)

{'a': 0, 'abandoned': 1, 'abhorrent': 2, 'absorb': 3, 'accomplished': 4, 'account': 5, 'actions': 6, 'activity': 7, 'adjusted': 8, 'adler': 9, 'admirable': 10, 'admirably': 11, 'admit': 12, 'again': 13, 'against': 14, 'akin': 15, 'all': 16, 'alternating': 17, 'always': 18, 'ambition': 19, 'among': 20, 'and': 21, 'any': 22, 'around': 23, 'as': 24, 'associated': 25, 'at': 26, 'atkinson': 27, 'attention': 28, 'attitude': 29, 'attracted': 30, 'away': 31, 'baker': 32, 'balanced': 33, 'be': 34, 'been': 35, 'behind': 36, 'bell': 37, 'between': 38, 'beyond': 39, 'blind': 40, 'bohemian': 41, 'books': 42, 'brilliantly': 43, 'brothers': 44, 'buried': 45, 'but': 46, 'by': 47, 'case': 48, 'chamber': 49, 'chest': 50, 'civil': 51, 'clasped': 52, 'clearing': 53, 'clues': 54, 'cocaine': 55, 'cold': 56, 'companion': 57, 'complete': 58, 'crack': 59, 'crime': 60, 'daily': 61, 'dark': 62, 'deeply': 63, 'delicate': 64, 'delicately': 65, 'desire': 66, 'distracting': 67, 'disturbing': 68, 'doings': 69, 'door'

Here we will consider a 3-gram model.

In [9]:
def n_gram_creator(text_ls:list):
    n_gram_ls=[]
    for i in range(2,len(text_ls)):
        ls=[text_ls[i-2],text_ls[i-1],text_ls[i]]
        n_gram_ls.append(ls)
    
    return n_gram_ls

#Checking the function.
text,_=extract_corpus_vocab(preprocess_text_file(['hello I"m Saikat','Who are you?']))  
n_gram_creator(text)

[['hello', 'i', 'm'],
 ['i', 'm', 'saikat'],
 ['m', 'saikat', 'who'],
 ['saikat', 'who', 'are'],
 ['who', 'are', 'you']]

Now we create a function that places the vocab index inplace of the words in the ngram list.

In [10]:
def ngram_to_number(sequence:list,vocab):
    vocab_dict=vocab_index_creator(vocab)
    ngram_numeric_ls=[]
    for ngram in sequence:
        ngram_row=[]
        for word in ngram:
            ngram_row.append(vocab_dict[word])

        ngram_numeric_ls.append(ngram_row)
    
    return ngram_numeric_ls
            
#Checking the function.
text,_=extract_corpus_vocab(preprocess_text_file(['hello I"m Saikat','Who are you?']))  
ngram=n_gram_creator(text)
print(f'Ngram list is',ngram)
print('\nNumeric version of Ngram list is:')
ngram_to_number(sequence=ngram,vocab=text)


Ngram list is [['hello', 'i', 'm'], ['i', 'm', 'saikat'], ['m', 'saikat', 'who'], ['saikat', 'who', 'are'], ['who', 'are', 'you']]

Numeric version of Ngram list is:


[[1, 2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 0], [5, 0, 6]]

Now we apply all the function we created to our original text data.

In [11]:
#Preprocessing the text data.
processed_data=preprocess_text_file(data)
#Creating a list of the vocab and its total length.
corpus_ls,vocab_length=extract_corpus_vocab(processed_data)
print(vocab_length)
#Creating vocabulary dictionary with word as the key and the index as the value.
vocab_dict=vocab_index_creator(corpus_ls)
#Creating a list of 3-gram sequence of the whole text corpus.
ngram_ls=n_gram_creator(corpus_ls)
#Converting the ngram list to integer list based on the word indexes.
ngram_numeric_ls=ngram_to_number(sequence=ngram_ls,vocab=corpus_ls)

print(f'Ngram list of the words:{ngram_ls[:5]}\n Ngram list made from the indexes of the words:{ngram_numeric_ls[:5]}')

313
Ngram list of the words:[['to', 'sherlock', 'holmes'], ['sherlock', 'holmes', 'she'], ['holmes', 'she', 'is'], ['she', 'is', 'always'], ['is', 'always', 'woman']]
 Ngram list made from the indexes of the words:[[283, 244, 125], [244, 125, 243], [125, 243, 140], [243, 140, 18], [140, 18, 307]]


In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [13]:
ngram_tensor=torch.tensor(ngram_numeric_ls,device=device)
ngram_tensor[:5]

tensor([[283, 244, 125],
        [244, 125, 243],
        [125, 243, 140],
        [243, 140,  18],
        [140,  18, 307]])

In [14]:
X_train=ngram_tensor[:,:-1]
Y_target=ngram_tensor[:,-1]
print(f'Training Input shape:{X_train.shape} and Training labl shape:{Y_target.shape}')

Training Input shape:torch.Size([311, 2]) and Training labl shape:torch.Size([311])


In [15]:
one_hot_dim_size=vocab_length
def one_hot_encoding(target):
    one_hot_ls=[]
    for idx in target:
        one_hot=np.zeros(one_hot_dim_size,dtype=np.float32)
        one_hot[idx]=1.
        one_hot_ls.append(one_hot)

    return one_hot_ls


In [16]:
Y_one_hot=one_hot_encoding(Y_target)
Y_tensor=torch.tensor(np.array(Y_one_hot),device=device)

Y_tensor[:5]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

Now we will create the LSTM subclass.

In [18]:
feature_dim=X_train.shape[1]
vocab_length=vocab_length
embedding_size=128
hidden_dim=128
dropout_rate=0.2
output_dim=one_hot_dim_size

In [31]:

class Next_Word_Lstm(nn.Module):
    def __init__(self, vocab_length, embedding_size, hidden_dim, feature_dim, output_dim, dropout_rate=0.2):
        super(Next_Word_Lstm, self).__init__()
        
        self.embedding = nn.Embedding(vocab_length, embedding_size)
        self.lstm = nn.LSTM(input_size=embedding_size, hidden_size=hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(hidden_dim*feature_dim , output_dim)

    def forward(self, X_tensor):
        embedded = self.embedding(X_tensor)
        output, _ = self.lstm(embedded)
        
        # Flatten or reshape the output from LSTM
#         output = output[:, -1, :]  # Get the last timestep's output
        output = output.reshape(output.size(0), -1)
        output = self.dropout(output)
        output = self.linear(output)

        return output

In [32]:
lstm_model=Next_Word_Lstm(vocab_length,embedding_size,hidden_dim,feature_dim,output_dim)
lstm_model.to(device)

Next_Word_Lstm(
  (embedding): Embedding(313, 128)
  (lstm): LSTM(128, 128, batch_first=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (linear): Linear(in_features=256, out_features=313, bias=True)
)

In [33]:
criterion=nn.CrossEntropyLoss()
criterion.to(device=device)

CrossEntropyLoss()

In [34]:
learning_rate=0.01
optimizer=torch.optim.Adam(lstm_model.parameters(),learning_rate)

In [ ]:
import torch
import torch.nn as nn

class Next_Word_Lstm(nn.Module):
    def __init__(self, vocab_length, embedding_size, hidden_dim, feature_dim, output_dim, dropout_rate=0.2):
        super(Next_Word_Lstm, self).__init__()
        
        self.embedding = nn.Embedding(vocab_length, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(hidden_dim * feature_dim, output_dim)

    def forward(self, X_tensor):
        embedded = self.embedding(X_tensor)
        output, _ = self.lstm(embedded)
        output = output.view(output.size(0), -1)
        output = self.dropout(output)
        output = self.linear(output)

        return output


In [35]:
epochs=250

for epoch in range(epochs):
    lstm_model.train()
    optimizer.zero_grad()
    Y_predict=lstm_model(X_train)
    loss=criterion(Y_predict,Y_target)
    if epoch%25==0:
        print(f'Next Word Model LSTM -> Epoch: {epoch}, Loss: {loss}')
    loss.backward()
    optimizer.step()


Next Word Model LSTM -> Epoch: 0, Loss: 5.7574944496154785
Next Word Model LSTM -> Epoch: 25, Loss: 1.0457306416356005e-05
Next Word Model LSTM -> Epoch: 50, Loss: 8.800732302916003e-07
Next Word Model LSTM -> Epoch: 75, Loss: 6.416586302293581e-07
Next Word Model LSTM -> Epoch: 100, Loss: 6.04477975230111e-07
Next Word Model LSTM -> Epoch: 125, Loss: 6.539250989590073e-07
Next Word Model LSTM -> Epoch: 150, Loss: 6.113777999416925e-07
Next Word Model LSTM -> Epoch: 175, Loss: 7.309690772672184e-07
Next Word Model LSTM -> Epoch: 200, Loss: 6.416589144464524e-07
Next Word Model LSTM -> Epoch: 225, Loss: 5.688307282980531e-07
